# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0831 11:20:46.134464  8112 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\in2ar\.conda\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0831 11:20:46.156407  8112 deprecation_wrapper.py:119] From C:\Users\in2ar\.conda\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{84169437507: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

84169437507

In [15]:
x.id

26129394246

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:26129394246 -> bob:84169437507]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
alice = sy.VirtualWorker(hook, id="alice")

In [22]:
x = th.tensor([1,2,3,4,5])

In [23]:
x_ptr = x.send(bob, alice)

In [24]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [25]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [26]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [27]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:33202579925 -> bob:95901920728]

In [29]:
y

(Wrapper)>[PointerTensor | me:29788110066 -> bob:71452302583]

In [30]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:13836740610 -> bob:26903901723]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:86851387108 -> bob:44860281753]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [35]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [36]:
z = (x + y).sum()

In [37]:
z.backward()

(Wrapper)>[PointerTensor | me:46019491543 -> bob:29304116066]

In [38]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [41]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [42]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [43]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [44]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{94155200528: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{10001206846: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:72976863363 -> bob:89793344650]

In [56]:
bob._objects

{89793344650: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{89793344650: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{89793344650: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{26929553947: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:77610735695 -> bob:1630905665]
Tensor B: tensor([1, 1, 1, 1, 1])

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:77610735695 -> bob:1630905665]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [67]:
from torch import nn, optim

In [68]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [69]:
# A Toy Model
model = nn.Linear(2,1)

In [70]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [71]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.3744)
tensor(0.3318)
tensor(0.1721)
tensor(0.1078)
tensor(0.0694)
tensor(0.0448)
tensor(0.0291)
tensor(0.0189)
tensor(0.0123)
tensor(0.0080)
tensor(0.0053)
tensor(0.0035)
tensor(0.0023)
tensor(0.0015)
tensor(0.0010)
tensor(0.0007)
tensor(0.0005)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


In [72]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [73]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [74]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [75]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [76]:
train()

tensor(0.4831, requires_grad=True)
tensor(1.0889, requires_grad=True)
tensor(0.4178, requires_grad=True)
tensor(0.6265, requires_grad=True)
tensor(0.2485, requires_grad=True)
tensor(0.3689, requires_grad=True)
tensor(0.1444, requires_grad=True)
tensor(0.2182, requires_grad=True)
tensor(0.0839, requires_grad=True)
tensor(0.1295, requires_grad=True)
tensor(0.0488, requires_grad=True)
tensor(0.0772, requires_grad=True)
tensor(0.0285, requires_grad=True)
tensor(0.0463, requires_grad=True)
tensor(0.0167, requires_grad=True)
tensor(0.0279, requires_grad=True)
tensor(0.0099, requires_grad=True)
tensor(0.0169, requires_grad=True)
tensor(0.0059, requires_grad=True)
tensor(0.0103, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0064, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0016, requires_grad=True)
tensor(0.0005, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [77]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [78]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [79]:
x = x.send(alice)

In [80]:
bob._objects

{99505161195: tensor([1, 2, 3, 4, 5])}

In [81]:
alice._objects

{68030251320: (Wrapper)>[PointerTensor | alice:68030251320 -> bob:99505161195]}

In [82]:
y = x + x

In [83]:
y

(Wrapper)>[PointerTensor | me:79483951448 -> alice:16921970659]

In [84]:
bob._objects

{99505161195: tensor([1, 2, 3, 4, 5]),
 17083776944: tensor([ 2,  4,  6,  8, 10])}

In [85]:
alice._objects

{68030251320: (Wrapper)>[PointerTensor | alice:68030251320 -> bob:99505161195],
 16921970659: (Wrapper)>[PointerTensor | alice:16921970659 -> bob:17083776944]}

In [86]:
jon = sy.VirtualWorker(hook, id="jon")

In [87]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [88]:
bob._objects

{57417706287: tensor([1, 2, 3, 4, 5])}

In [89]:
alice._objects

{4473998183: (Wrapper)>[PointerTensor | alice:4473998183 -> bob:57417706287]}

In [90]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:4473998183 -> bob:57417706287]

In [91]:
bob._objects

{57417706287: tensor([1, 2, 3, 4, 5])}

In [92]:
alice._objects

{}

In [93]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [94]:
bob._objects

{}

In [95]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [96]:
bob._objects

{30378399939: tensor([1, 2, 3, 4, 5])}

In [97]:
alice._objects

{51709488345: (Wrapper)>[PointerTensor | alice:51709488345 -> bob:30378399939]}

In [98]:
del x

In [99]:
bob._objects

{}

In [100]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [101]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [102]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [103]:
bob._objects

{51208588197: tensor([1, 2, 3, 4, 5])}

In [104]:
alice._objects

{}

In [105]:
x.move(alice)

(Wrapper)>[PointerTensor | me:40412095949 -> alice:40412095949]

In [106]:
bob._objects

{}

In [107]:
alice._objects

{40412095949: tensor([1, 2, 3, 4, 5])}

In [108]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [109]:
bob._objects

{35652938003: tensor([1, 2, 3, 4, 5])}

In [110]:
alice._objects

{40412095949: tensor([1, 2, 3, 4, 5]),
 50243632229: (Wrapper)>[PointerTensor | alice:50243632229 -> bob:35652938003]}

In [111]:
x.remote_get()

(Wrapper)>[PointerTensor | me:51485673474 -> alice:50243632229]

In [112]:
bob._objects

{}

In [113]:
alice._objects

{40412095949: tensor([1, 2, 3, 4, 5]), 50243632229: tensor([1, 2, 3, 4, 5])}

In [114]:
x.move(bob)

(Wrapper)>[PointerTensor | me:51485673474 -> bob:51485673474]

In [115]:
x

(Wrapper)>[PointerTensor | me:51485673474 -> bob:51485673474]

In [116]:
bob._objects

{51485673474: tensor([1, 2, 3, 4, 5])}

In [117]:
alice._objects

{40412095949: tensor([1, 2, 3, 4, 5])}